In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/cs330_project/ama_logs/final_runs/super_glue_wsc

/content/drive/.shortcut-targets-by-id/12R0IzSvJ39kGSsKxOP2KtyB8WDBpWfaQ/cs330_project/ama_logs/final_runs/super_glue_wsc


In [3]:
import json
from collections import defaultdict

In [26]:
to_compare = {
    "k 1,7,3": "/content/drive/MyDrive/cs330_project/ama_logs/v3_final_runs/super_glue_wsc/EleutherAI_gpt-neo-1.3B_k_per_chain=1,7,3_12092022.json",
    "k 3,7,3": "/content/drive/MyDrive/cs330_project/ama_logs/v3_final_runs/super_glue_wsc/EleutherAI_gpt-neo-1.3B_k_per_chain=3,7,3_12092022.json"
}

In [27]:
def open_log(path):
    f = open(path)
    log = json.load(f)
    return log

def most_common(lst):
    return max(set(lst), key=lst.count)

def collect_outputs(to_compare):
    logs = []
    names = []
    for name,path in to_compare.items():
        log = open_log(path)
        logs.append(log)
        names.append(name)

    extractions = []
    questions = []
    predictions = []
    answers = []
    gold = [""] * len(logs[0])
    passages = [""] * len(logs[0])
    for i in range(len(logs[0])):
      cur_extractions = defaultdict(list)
      cur_questions = defaultdict(list)
      cur_predictions = defaultdict(list)
      cur_answers = defaultdict(list)
      for idx, name in enumerate(names):
        cur_log = logs[idx][str(i)]
        prompts = cur_log["prompts"]
        name = names[idx]
        
        for j in range(len(prompts)):
          cur_extractions[name].append(cur_log['prompts'][j][1].split('\n\n')[-1].split("\n")[0][7:])
          cur_questions[name].append(cur_log['prompts'][j][2].split('\n\n')[-1].split("\n")[-2][10:])
        cur_predictions[name].extend(cur_log["preds_boost"])
        cur_answers[name].extend(cur_log["answers_boost"])
        
        gold[i] = cur_log["gold"]
        passages[i] = cur_log["example"]
      
      # add the dicts at the current index
      extractions.append(cur_extractions)
      questions.append(cur_questions)
      predictions.append(cur_predictions)
      answers.append(cur_answers)

    return (extractions, questions, predictions, gold, passages,answers,names)

def find_different_outputs(edited_logs, num_show=5):
  extractions, questions, predictions, gold, passages, answers,names = edited_logs
  count = 0
  for i in range(len(extractions)):
    cur_predictions = map(most_common,predictions[i].values())
    # print(set(cur_predictions))
    if len(set(cur_predictions)) > 1:
      count += 1
      print(f'passage: {passages[i]}')
      print(f'gold label (at idx = {i}): {gold[i]}')
      for name in names:
        print(f'run name: {name}')
        print(f'extractions: {extractions[i][name]}')
        print(f'question: {questions[i][name]}')
        print(f'answers: {answers[i][name]}')
        print(f'predictions: {predictions[i][name]}')
        print()
      print('-'*50)
    
    if count == num_show: break

In [28]:
edited_logs=collect_outputs(to_compare)

In [29]:
find_different_outputs(edited_logs, 5)

passage: Larry , a timid teen-ager, lives with his widowed mother in a Brooklyn housing project. Larry Larry's father , a gang leader, was shot to death; his father's disciple, Antonio , takes Larry under his wing, and quickly molds him into a drug runner.
gold label (at idx = 13): True
run name: k 1,7,3
extractions: ['his father', 'his father', 'his also molds Larry into a drug runner']
question: ['Whose father?', 'Whose father?', 'Whose drug runner?']
answers: ['Antonio', 'Antonio', 'Larry']
predictions: ['True', 'True', 'False']

run name: k 3,7,3
extractions: ['his father', 'his is so short', 'his father Antonio']
question: ['Whose father?', 'Whose is so short?', 'Whose father Antonio?']
answers: ['Antonio', 'Larry', 'Larry']
predictions: ['True', 'False', 'False']

--------------------------------------------------
passage: Larry , a timid teen-ager, lives with his widowed mother in a Brooklyn housing project. Larry Larry's father , a gang leader, was shot to death; his father's d

In [17]:
extractions, questions, predictions, gold, passages, answers,names = edited_logs

In [10]:
from collections import Counter
print(Counter(gold))
print(66/(66+38))
print(38/(66+38))

Counter({'False': 66, 'True': 38})
0.6346153846153846
0.36538461538461536
